## Preprocessing

In [1]:
# Simple place to select columns which should be dropped below.  Just add here and run all.

DROP_COLUMNS = ['EIN','STATUS','SPECIAL_CONSIDERATIONS']

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Note:
#### Features:
APPLICATION_TYPE 	AFFILIATION 	CLASSIFICATION 	USE_CASE 	ORGANIZATION 	STATUS 	INCOME_AMT 	SPECIAL_CONSIDERATIONS 	ASK_AMT

#### Target: 
IS_SUCCESSFUL

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# application_df = application_df.drop(columns=['EIN','NAME'])

In [4]:
# Determine the number of unique values in each column.
for c in application_df.columns:
    print(f'{c:<30}{len(application_df[c].unique())}')

EIN                           34299
NAME                          19568
APPLICATION_TYPE              17
AFFILIATION                   6
CLASSIFICATION                71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                       8747
IS_SUCCESSFUL                 2


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at <BLANK> value counts for binning
application_df.ASK_AMT.value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# This cell generalizes replacement cutoffs for any column listed, as well as a corresponding cutoff level.
# To use elsewhere, check the column names within the dataframe.
# CHECK DATAFRAME NAME

column_list = ['APPLICATION_TYPE', 'CLASSIFICATION']
# OPTIMIZATION NOTE: CHANGING CUTOFF VALUES
cutoff_list = [200               , 150             ]
for i in range(len(column_list)):
    column = column_list[i]
    cutoff = cutoff_list[i]
    # list of unique values within single column
                    ########## DATAFRAME
    col_type_list = application_df[column].unique().tolist()
    col_type_list
#     print(len(col_type_list))
    
    # list of counts of each unique value within single column
    col_amt_list = []
    for col in col_type_list:
                                ########## DATAFRAME       ########## DATAFRAME
        col_amt_list.append(len(application_df[column].loc[application_df[column]==col]))
    col_amt_list
#     print(len(col_amt_list))

    reduced_col_list = []
    for i in range(len(col_type_list)):
        if col_amt_list[i] <= cutoff:
            reduced_col_list.append(col_type_list[i])
    reduced_col_list

    
    for name in reduced_col_list:
        ########## DATAFRAME     ########## DATAFRAME
        application_df[column] = application_df[column].replace({
            name: 'Other'
        })
    print(column)
          ########## DATAFRAME
    print(application_df[column].value_counts())
    print('#'*30)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64
##############################
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64
##############################


In [9]:
# TEST REMOVING COLUMNS
dropped_df = application_df.drop(columns=DROP_COLUMNS)

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(dropped_df)
application_dummies

,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,NAME_146TH ALUMNI ASSOCIATION,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays
X_dummies = application_dummies.drop(columns=['IS_SUCCESSFUL'])
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape
X_t_num_cols = X_train_scaled.shape[1]
X_t_num_cols

19611

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu', input_dim = X_t_num_cols))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                588360    
                                                                 
 dense_1 (Dense)             (None, 30)                930       
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 589,321
Trainable params: 589,321
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Weight Saver Callback Function
from keras.callbacks import Callback
class WeightsSaver(Callback):
  def __init__(self, N):
    self.N = N
    self.epoch = 0

  def on_epoch_end(self, epoch, logs={}):
    if self.epoch % self.N == 0:
      name = 'weights%08d.h5' % self.epoch
      folder = '../Models/Weights-Optimized/'
      self.model.save_weights(folder + name)
    self.epoch += 1


# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=50, callbacks=[WeightsSaver(5)])

Epoch 1/50
804/804 [==============================] - 4s 4ms/step - loss: 0.4870 - accuracy: 0.7668
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1718 - accuracy: 0.9427
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1046 - accuracy: 0.9605
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0962 - accuracy: 0.9608
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0930 - accuracy: 0.9617
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0919 - accuracy: 0.9619
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0910 - accuracy: 0.9623
Epoch 8/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0894 - accuracy: 0.9637
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0893 - accuracy: 0.9632
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0884 - accuracy: 0.9637

In [16]:
# ATTEMPT 2
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu', input_dim = X_t_num_cols))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="tanh"))
# CHANGED UNITS HERE

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                588360    
                                                                 
 dense_4 (Dense)             (None, 15)                465       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 588,841
Trainable params: 588,841
Non-trainable params: 0
_________________________________________________________________


In [17]:
# ATTEMPT 2
# Weight Saver Callback Function
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=50, callbacks=[WeightsSaver(5)])

Epoch 1/50
804/804 [==============================] - 4s 4ms/step - loss: 0.4874 - accuracy: 0.7666
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.2027 - accuracy: 0.9294
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1088 - accuracy: 0.9595
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0982 - accuracy: 0.9615
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0946 - accuracy: 0.9623
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0927 - accuracy: 0.9625
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0912 - accuracy: 0.9630
Epoch 8/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0906 - accuracy: 0.9631
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0899 - accuracy: 0.9631
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.0889 - accuracy: 0.9634

In [22]:
X_t_num_cols

19611

In [18]:
# ATTEMPT 3: PREVIOUS ATTEMPTS LOWERED SCORE.  ATTEMPTING ORIGINAL SETTINGS WITH DIFFERENT OPTIMIZER.
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='leaky_relu', input_dim = X_t_num_cols))

# Second hidden layer
x = tf.ones([3,5])


nn.add(tf.keras.layers.Dense(units=100, activation="elu"))
# nn.add(tf.keras.layers.Dense(units=100, activation="selu"))



# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               1961200   
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,971,401
Trainable params: 1,971,401
Non-trainable params: 0
_________________________________________________________________


In [19]:
# ATTEMPT 3
# Weight Saver Callback Function
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=50, callbacks=[WeightsSaver(5)])

Epoch 1/50
804/804 [==============================] - 6s 7ms/step - loss: 0.4901 - accuracy: 0.7653
Epoch 2/50
804/804 [==============================] - 5s 7ms/step - loss: 0.1146 - accuracy: 0.9593
Epoch 3/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0991 - accuracy: 0.9603
Epoch 4/50
804/804 [==============================] - 6s 7ms/step - loss: 0.0962 - accuracy: 0.9602
Epoch 5/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0944 - accuracy: 0.9605
Epoch 6/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0929 - accuracy: 0.9620
Epoch 7/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0925 - accuracy: 0.9610
Epoch 8/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0911 - accuracy: 0.9624
Epoch 9/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0907 - accuracy: 0.9619
Epoch 10/50
804/804 [==============================] - 5s 7ms/step - loss: 0.0897 - accuracy: 0.9627

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6802 - accuracy: 0.6858 - 919ms/epoch - 3ms/step
Loss: 0.6802119612693787, Accuracy: 0.6858308911323547


In [21]:
# Export our model to HDF5 file
nn.save("../Models/AlphabetSoupCharity_Optimization.h5")